In [1]:
import os, sys
from os.path import dirname

root_path = os.path.abspath(os.getcwd())
if not root_path in sys.path: sys.path.insert(0, root_path)
    
print(sys.path)

['D:\\GitWork\\mnist', 'C:\\opt\\miniconda3\\envs\\py37\\python37.zip', 'C:\\opt\\miniconda3\\envs\\py37\\DLLs', 'C:\\opt\\miniconda3\\envs\\py37\\lib', 'C:\\opt\\miniconda3\\envs\\py37', '', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages\\win32', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages\\win32\\lib', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages\\Pythonwin', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Morpheus\\.ipython']


In [2]:
# If you want to download MNIST data by torchvision function

# your_data_path = ''
# torchvision.datasets.MNIST(your_data_path, train=True, transform=None, target_transform=None, download=True)

In [3]:
# You also can build dataset from the download files of http://yann.lecun.com/exdb/mnist/

import gzip
import pprint
import shutil

RootPath_Code = 'D:\\GitWork\\mnist\\'
RootPath_Data = 'D:\\GitWork\\mnist\\data\\'

Dir_Processed = 'processed'
Dir_Raw = 'raw'

Fname_Train = 'training.pt'
Fname_Test  = 'test.pt'

Resources = [
    "train-images-idx3-ubyte.gz", 
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"
]

path_processed = os.path.join(RootPath_Data, Dir_Processed)
path_raw = os.path.join(RootPath_Data, Dir_Raw)

if not os.path.exists(path_processed): os.makedirs(path_processed)

abspath_trainset = os.path.join(path_processed, Fname_Train)
abspath_testset  = os.path.join(path_processed, Fname_Test)

isTrainDataExist = os.path.exists(abspath_trainset)
isTestDataExist  = os.path.exists(abspath_testset)

print("'{}' exist: {}".format(abspath_trainset, isTrainDataExist))
print("'{}' exist: {}".format(abspath_testset, isTestDataExist))

# If all files exist then return 'True' else 'False'
def isExistAllFiles(path, files, debug=False):
    isExistList = [os.path.exists(os.path.join(path, f)) for f in files]
    if debug: print(isExistList)
    return all(isExistList)
                    
isExistResources = isExistAllFiles(path_raw, Resources)
print(isExistResources)

datafiles = [f.replace('.gz', '') for f in Resources]
isExistDataFiles = isExistAllFiles(path_raw, datafiles)
print(isExistDataFiles)
    
if not isExistDataFiles and isExistResources:
    print('Data files not exist, decompressing...')
    for f in Resources:
        inFile  = os.path.join(path_raw, f)
        outFile = os.path.join(path_raw, f.replace('.gz', ''))
        with gzip.open(inFile, 'r') as f_in, open(outFile, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)      
    print('Data files decompressed')

filelist = [f for f in os.listdir(path_raw)]
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(filelist)

'D:\GitWork\mnist\data\processed\training.pt' exist: True
'D:\GitWork\mnist\data\processed\test.pt' exist: True
True
True
[ 't10k-images-idx3-ubyte',
  't10k-images-idx3-ubyte.gz',
  't10k-labels-idx1-ubyte',
  't10k-labels-idx1-ubyte.gz',
  'train-images-idx3-ubyte',
  'train-images-idx3-ubyte.gz',
  'train-labels-idx1-ubyte',
  'train-labels-idx1-ubyte.gz']


In [4]:
from __future__ import print_function, division  

import codecs
import numpy as np
import time


def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)

def open_maybe_compressed_file(path):
    """Return a file object that possibly decompresses 'path' on the fly.
       Decompression occurs when argument `path` is a string and ends with '.gz' or '.xz'.
    """
    if not isinstance(path, torch._six.string_classes):
        return path
    if path.endswith('.gz'):
        import gzip
        return gzip.open(path, 'rb')
    if path.endswith('.xz'):
        import lzma
        return lzma.open(path, 'rb')
    return open(path, 'rb')

def read_sn3_pascalvincent_tensor(path, strict=True):
    """Read a SN3 file in "Pascal Vincent" format (Lush file 'libidx/idx-io.lsh').
       Argument may be a filename, compressed filename, or file object.
    """
    # typemap
    if not hasattr(read_sn3_pascalvincent_tensor, 'typemap'):
        read_sn3_pascalvincent_tensor.typemap = {
            8: (torch.uint8, np.uint8, np.uint8),
            9: (torch.int8, np.int8, np.int8),
            11: (torch.int16, np.dtype('>i2'), 'i2'),
            12: (torch.int32, np.dtype('>i4'), 'i4'),
            13: (torch.float32, np.dtype('>f4'), 'f4'),
            14: (torch.float64, np.dtype('>f8'), 'f8')
        }
        
    # read
    with open_maybe_compressed_file(path) as f:
        data = f.read()

    # parse
    magic = get_int(data[0:4])
    nd = magic % 256
    ty = magic // 256
    assert nd >= 1 and nd <= 3
    assert ty >= 8 and ty <= 14
    m = read_sn3_pascalvincent_tensor.typemap[ty]
    s = [get_int(data[4 * (i + 1): 4 * (i + 2)]) for i in range(nd)]
    parsed = np.frombuffer(data, dtype=m[1], offset=(4 * (nd + 1)))
    assert parsed.shape[0] == np.prod(s) or not strict
    return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)

def read_image_file(path):
    with open(path, 'rb') as f:
        x = read_sn3_pascalvincent_tensor(f, strict=False)
    assert(x.dtype == torch.uint8)
    assert(x.ndimension() == 3)
    return x

def read_label_file(path):
    with open(path, 'rb') as f:
        x = read_sn3_pascalvincent_tensor(f, strict=False)
    assert(x.dtype == torch.uint8)
    assert(x.ndimension() == 1)
    return x.long()


def createProcessedDataFiles(path, inFiles, outFiles):

    training_set = (
        read_image_file(os.path.join(path, inFiles[0])),
        read_label_file(os.path.join(path, inFiles[1]))
    )

    test_set = (
        read_image_file(os.path.join(path, inFiles[2])),
        read_label_file(os.path.join(path, inFiles[3]))
    )
    
    with open(os.path.join(path_processed, outFiles[0]), 'wb') as f:
        torch.save(training_set, f)

    with open(os.path.join(path_processed, outFiles[1]), 'wb') as f:
        torch.save(test_set, f)
    
    return (0)


inFiles  = [f.replace('.gz', '') for f in Resources]
outFiles = [Fname_Train, Fname_Test]

isExistPtFiles = isExistAllFiles(path_processed, outFiles)
print(isExistPtFiles)
if not isExistPtFiles:
    createProcessedDataFiles(path_raw, inFiles, outFiles)

for f in outFiles:
    isExist = os.path.exists(os.path.join(path_processed, f))
    print('{} exist: {}'.format(f, isExist))


True
training.pt exist: True
test.pt exist: True


In [5]:
import matplotlib.pyplot as plt

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [6]:
# Load dataset of images and labels by torch.load()

# Load entire dataset
images, labels = torch.load(os.path.join(path_processed, 'training.pt'))
print('train images shape: ', list(images.shape))
print('train labels shape: ', list(labels.shape))
print()

images, labels = torch.load(os.path.join(path_processed, 'test.pt'))
print('test images shape: ', list(images.shape))
print('test labels shape: ', list(labels.shape))

train images shape:  [60000, 28, 28]
train labels shape:  [60000]

test images shape:  [10000, 28, 28]
test labels shape:  [10000]


In [7]:
# Load dataset of images and labels by torch.dataset()

# class myMnistDataset(Dataset):

#     def __init__(self, path, phase='train', transform=None):
#         self.transform = transform
        
#         fname = 'training.pt' if phase == 'train' else 'test.pt'
#         imgs, lbls = torch.load(os.path.join(path, fname))
#         self.len = imgs.shape[0]
        
#         imgs = imgs.type(torch.float).view(self.len, -1)
#         print(imgs.shape)
#         print(imgs[0])
        
#         lbls = lbls.type(torch.float).reshape(self.len, -1)
#         print(lbls.shape)
#         print(lbls[0])
        
#         self.imgs = imgs
#         self.lbls = lbls
    
#     def __len__(self):
#         return self.len
    
#     def __getitem__(self, idx):
#         return (self.imgs[idx], self.lbls[idx])
    
#     def shape(self, phase='train'):
#         shape = torch.tensor([-1, -1, -1])
#         if phase == 'train': 
#             shape = self.imgs.shape
#         elif phase == 'test' :
#             shape = self.lbls.shape
#         return shape

class myMnistDataset(Dataset):
    
    def __init__(self, transform=None):
        self.transform = transform
        csv_path = os.path.join('D:\\GitWork\\mnist\\csvFiles\\', 'mnist_train.csv')
        xy = np.loadtxt(csv_path, delimiter=',', dtype=np.float32)
        
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 1:])
        self.y_data = torch.from_numpy(xy[:, [0]])

    def __getitem__(self, index):
        return (self.x_data[index], self.y_data[index])

    def __len__(self):
        return self.len
    

def imgshow(img, label, phase='train'):
    str_label = str(label.numpy()[0])
    plt.title("{}.label = {}".format(phase, str_label))
    img = img.reshape(28,28)

    plt.imshow(img, cmap='binary')
    plt.show()
    
# ds_train = myMnistDataset(path_processed)
# (trainX, trainY) = ds_train[0]
# imgshow(trainX, trainY)


In [8]:
# csv_path = os.path.join('D:\\GitWork\\mnist\\csvFiles\\', 'mnist_train.csv')
# xy = np.loadtxt(csv_path, delimiter=',', dtype=np.float32)

# len = xy.shape[0]
# x_data = torch.from_numpy(xy[:, 1:])
# y_data = torch.from_numpy(xy[:, [0]])

# print(x_data.shape)
# print(x_data[0])
# print(y_data.shape)
# print(y_data[0])

In [9]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

print('GPU State:', device)

GPU State: cuda:0


In [10]:
# Prepare dataset and dataloader

# Create train/test transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3801,))
])

# Create datasets
trainSet = myMnistDataset(transform=transform)
# validSet = myMnistDataset(transform=transform)

# Create dataloaders
BATCH_SIZE = 64
# NUM_WORKERS = 2

trainLoader = DataLoader(
    dataset=trainSet, batch_size=BATCH_SIZE, shuffle=True
)

# valid_loader = DataLoader(
#     dataset=validSet, batch_size=BATCH_SIZE, shuffle=False
# )

In [11]:
# Model
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(Net, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=784, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=10),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, input):
        return self.main(input)

input_size = 784
hidden_size1 = 128
hidden_size2 = 64
num_classes = 10
    
net = Net(input_size, hidden_size1, hidden_size2, num_classes).to(device)
print(net)

Net(
  (main): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): LogSoftmax()
  )
)


In [12]:
import torch.optim as optim
from torch.autograd import Variable

epochs = 3
learning_rate = 0.002
criterion = nn.NLLLoss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

# Train
for epoch in range(epochs):
    running_loss = 0.0

    for i, (images, labels) in enumerate(trainLoader, 0):
        
        images = Variable(images.view(-1, 28*28).cuda())
        labels = Variable(labels.cuda())

        print(images.dtype)
        print(labels.dtype)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Foward + backward + optimize
        outputs = net(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if (i+i) % BATCH_SIZE == 0 or (i+1) == len(trainLoader):
            print('[%d/%d, %d/%d] loss: %.3f' % (epoch+1, epochs, i+1, len(trainLoader), running_loss/2000))

print('Training Finished.')

torch.float32
torch.float32
torch.int64


RuntimeError: multi-target not supported at C:/cb/pytorch_1000000000000/work/aten/src\THCUNN/generic/ClassNLLCriterion.cu:18